# StopIteration

You CAN **NOT** raise a StopIteration from inside a Generator to catch it in the outer caller.

* [exception StopIteration](https://docs.python.org/3/library/exceptions.html#StopIteration)

> Changed in version 3.7: Enable PEP 479 for all code by default: **a StopIteration error raised in a generator is transformed into a RuntimeError**.

* [PEP 479 -- Change StopIteration handling inside generators](https://www.python.org/dev/peps/pep-0479/)

> Currently, **StopIteration raised accidentally inside a generator function will be interpreted as the end of the iteration** by the loop construct driving the generator.

* [Don't raise StopIteration, raise something sane like ValueError](https://stackoverflow.com/a/63163984/4281353)

>  StopIteration serves a very specific purpose (to allow a __next__ method to indicate iteration is complete), and reusing it for other purposes will cause problems, as you've seen. 
> 
> The conversion to RuntimeError here is saving you; if Python hadn't done that, the **generator would have silently stopped iterating (StopIteration is swallowed silently, causing iteration to end without propagating the exception; you'd never catch it anyway)**.

## Example



In [ ]:
from typing import (
    Generator,
    Iterable,
    List
)
import sys
import pathlib
from itertools import islice

import numpy as np


def take(n: int, iterable: Iterable) -> List:
    taken = list(islice(iterable, 0, n))
    if len(taken) > 0:
        return taken
    else:
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("Raising StopIteration")
        raise StopIteration("Nothing to take")     # <----- Raise StopIteration


def file_line_stream(path: str) -> Generator[str, None, None]:
    if not pathlib.Path(path).is_file():
        raise FileNotFoundError(f"file {path} does not exist or non file")
    try:
        _file = pathlib.Path(path)
        with _file.open() as f:
            for line in f:
                yield line.rstrip()
    except (IOError, FileNotFoundError) as e:
        raise RuntimeError("Read file failure") from e


def get_sentences(path_to_file, num_sentences):
    counter = 0
    stream = file_line_stream(path_to_file)
    try:
        while True:
            _lines = take(num_sentences, stream)
            print("------------------------------------------------------------------------")
            print(f"count {counter}:  yielding [{_lines[0]}]\n")
            yield np.array(_lines)
            counter += 1
    finally:
        stream.close()


def test_word2vec():
    path_to_input = "test_has_two_lines.txt"
    generator = get_sentences(path_to_file=path_to_input, num_sentences=100)
    for i in range(10):
        # ----------------------------------------------------------------------
        # Want to catch the StopIteration raised from within the
        # generator call stack.
        # ----------------------------------------------------------------------
        try:
            next(generator)
        except StopIteration as e:
            print("OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO")
            print("Please come to here")
            generator.close()

        except Exception as e:
            print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
            print("Please do not come here !!!!!!!!!: %s", sys.exc_info()[0])
            generator.close()
            raise e


if __name__ == '__main__':
    test_word2vec()


## Result

```
------------------------------------------------------------------------
count 0:  yielding [previous monthly <unk> of the major market index futures and standard & poor 's <unk> index options have produced spectacular volatility]

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Raising StopIteration

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Please do not come here !!!!!!!!!: %s <class 'RuntimeError'>
Traceback (most recent call last):
  File "test_050_word2vec.py", line 40, in get_sentences
    _lines = take(num_sentences, stream)
  File "test_050_word2vec.py", line 20, in take
    raise StopIteration("Nothing to take")     # <----- Raise StopIteration
StopIteration: Nothing to take

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "test_050_word2vec.py", line 72, in <module>
    test_word2vec()
  File "test_050_word2vec.py", line 68, in test_word2vec
    raise e
  File "test_050_word2vec.py", line 58, in test_word2vec
    next(generator)

RuntimeError: generator raised StopIteration    <---------- RuntimeError
```